In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('dark_background')
background_color = tuple(np.array([66, 68, 69])/255)
plt.rcParams['axes.facecolor'] = background_color
plt.rcParams['figure.facecolor'] = background_color

# Idea

Our linear layer is linearized and made positive-definite using low dimentional factorization with "residual" connection, $U^TU$ is being kept lipschitz with constant smaller than one.

$$ m >> n \qquad U \in \mathbb{R}^{n \times m} \qquad U^TU$$
$$ l_{U}(x) = \sigma((U^TU + I)x) = \sigma( U^T (Ux) + x ) $$


In [127]:
x = torch.rand(100)
a = 10
U = torch.rand(10,100)/a
V = torch.rand(10,100)/a
M = (U.T @ U) - (V.T @ V) + torch.eye(100)
M = (U.T @ U) + torch.eye(100)

In [128]:
torch.linalg.eigh(M).eigenvalues

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0491, 1.0547, 1.0691, 1.0705, 1.0796, 1.0860, 1.0923, 1.1128, 1.1250,
        3.5749])

In [133]:
torch.linalg.matrix_norm(U.T @ U, ord=2)

tensor(2.5749)

In [130]:
torch.svd(V).S.pow(2)

tensor([2.6255, 0.1227, 0.1082, 0.0959, 0.0912, 0.0807, 0.0682, 0.0629, 0.0600,
        0.0531])

In [131]:
torch.linalg.matrix_norm(V, ord=2).pow(2)

tensor(2.6255)

In [132]:
torch.svd(V).S.pow(2) - 1

tensor([ 1.6255, -0.8773, -0.8918, -0.9041, -0.9088, -0.9193, -0.9318, -0.9371,
        -0.9400, -0.9469])